## Setup

### Imports

In [1]:
import plotly.graph_objects as go
import numpy as np
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import seaborn as sns
from math import isnan
import matplotlib.pyplot as plt

In [2]:
sns.set_style('darkgrid')

### Definitions

In [3]:
path_data_dir = '../data/'

In [4]:
list_assets = ["PETR3.SA","PRIO3.SA", "VALE3.SA", "GGBR3.SA", "ABCB4.SA", "ITUB3.SA", "FLRY3.SA", "RADL3.SA"]

In [5]:
relevant_cols = ['Date', 'Close', 'Volume']

In [6]:
# simple moving average 
window_size = 21

###  Prepare the data

In [7]:
def expand_days(
    data
):

    # get all dates in the range
    date_range = pd.date_range(start=data['Date'].min(), end=data['Date'].max())

    # create df with all dates
    full_dates_df = pd.DataFrame({'Date': date_range})

    # merge dataframe with all dates and original 
    data = full_dates_df.merge(data, on='Date', how='left')
    
    return data

In [8]:
def calculate_features(data):
    
    data['diff_close_mean'] = data.Close - data.SMA_21
    data['diff_close_mean_z_score'] = data.diff_close_mean / data.MSTD_21
    data['diff_close_mean_z_score'] = data['diff_close_mean_z_score'].fillna(0)
    data['meta'] = data.diff_close_mean_z_score.apply(int)
    
    return data 

#### create dataframe

In [9]:
for asset in tqdm(list_assets):
    
    # get data
    data =  pd.read_excel(path_data_dir + f'raw/price_history_{asset.replace('.', '_')}.xlsx')[relevant_cols]
    
    data[f'SMA_{window_size}'] = data[f'Close'].rolling(window = window_size).mean()
    data[f'MSTD_{window_size}'] = data[f'Close'].rolling(window = window_size).std()
    
    # remove n first rows that not enter in the SMA
    data = data.iloc[window_size - 1:,:]
    
    # expand days
    data = expand_days(data)
    
    
    cols_to_ffill = [f'SMA_{window_size}',f'MSTD_{window_size}'] 
    data[cols_to_ffill] = data[cols_to_ffill].ffill()
    
    # create column with day of week
    data['Day_of_week'] = data.Date.dt.day_of_week
    
    # tag weekend
    data['Weekend'] = data.Day_of_week.apply(lambda x: 1 if x in [5,6] else 0)
    
    # data invalid days 
    data['Invalid_Days'] = data.Close.apply(lambda x: 1 if isnan(x) else 0)
    
    data.set_index('Date', inplace=True)
    
    data_filled = data.ffill()
    data_interpolate = data.interpolate()
    
    data_filled = calculate_features(data_filled)
    data_interpolate = calculate_features(data_interpolate)
    
    data_filled.to_csv(path_data_dir + f'processed/price_history_{asset.replace('.', '_')}_meta_dataset_ffill.csv')
    data_interpolate.to_csv(path_data_dir + f'processed/price_history_{asset.replace('.', '_')}_meta_dataset_interpolate.csv')

100%|██████████| 8/8 [00:02<00:00,  3.62it/s]
